# Aproximação de uma função (Regressão)

Para este trabalho a biblioteca `anfis-pytorch` foi utilizada para a criação de uma rede ANFIS.


In [1]:
import sys
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch
import anfis
from matplotlib.ticker import FormatStrFormatter
from torch.utils.data import TensorDataset, DataLoader
from membership import BellMembFunc, make_bell_mfs, GaussMembFunc, make_gauss_mfs
from experimental import train_anfis, test_anfis

dtype = torch.float

%matplotlib inline


ModuleNotFoundError: No module named 'torch'

A função `plot` criada abaixo, apenas faz o gráfico com pontos vermelhos (caso não sejam dados outros parâmetros) de um dado conjuntos de pares ordenados. Na verdade, os pares são feitos a partir de dois arrays, o primeiro com os valores das abscissas e o segundo com o das ordenadas.

In [ ]:
def plot(x, y, plot_color = 'red', plot_label = 'Dados de entrada'):
    plt.plot(x,y,'o', color = plot_color, label = plot_label)
    plt.legend(loc="upper right")
    plt.show()

A função `seno` apenas aplica a função seno da biblioteca `numpy` com ou sem um dado ruído.

In [ ]:
def seno(x, r = 0):
    y = np.sin(x)
    if (r != 0):
      y = y + np.random.normal(0, r, y.shape)
    return y

A seguinte função devolve um `DataLoader` ou seja, um objeto iterável da biblioteca `torch` que guarda os dados sobre as entradas e saídas de uma dada função $f(\cdot)$.

In [ ]:
def cria_dados(f, batch_size=10, ruido = 0, inicio =0, fim=2*np.pi):
    pts = torch.arange(inicio, fim, 0.1)
    x = torch.tensor(list(itertools.product(pts)), dtype=dtype)
    y = [[f(p, ruido)] for p in x]
    y = torch.tensor(y, dtype=dtype)
    plot(x,y)
    td = TensorDataset(x, y)
    return DataLoader(td, batch_size=batch_size, shuffle=True)

A função abaixo gera o modelo inicial de uma ANFIS que tem como função fuzzificadora a função Gaussiana.

In [ ]:
def cria_modelo(sigma = 0.1, mu_list = [1.0, 2.0]):
    definicoes_entrada = [('x0', make_gauss_mfs(sigma, mu_list))]
    variavel_saida = ['y0']
    anf_inicial = anfis.AnfisNet('Jang\'s example 1', invardefs = definicoes_entrada, outvarnames= variavel_saida)
    return anf_inicial

## Testes

### Teste 1: sem ruído e três partes

Como se pode ver, ao considerar uma função seno sem ruído a ANFIS tem um desempenho quase perfeito

In [ ]:
modelo = cria_modelo(mu_list = [1.0, 2.0, 3.0])
train_data = cria_dados(seno)
train_anfis(modelo, train_data, 100, True) 

### Teste 2: com ruído e duas partes

Ao se inserir um ruído nos dados e diminuir as funções de pertinência de três para duas, é possível notar que a porcentagem de erro aumenta consideravelmente, se mantendo na casa dos $200\%$.

In [ ]:
##Teste 2: ruído = 0.1; sigma = 10; duas funções de pertinência, 
##Ao adicionar um ruído de 0.1, a função seno continua com o seu formato, porém
##ao deixar o sigma com um número médio, a porcentagem de erro fica alto e constante.
##e além disso, o resultado final não fica muito bom e, podemos destacar que utilizamos 
##apenas duas funções de pertinência.
modelo = cria_modelo(10)
train_data = cria_dados(seno, ruido = 0.1)
train_anfis(modelo, train_data, 100, True) 

### Teste 3: com ruído, base larga e duas partes

Diferentemente do último teste, as funções de pertinência deste têm um sigma ($\sigma$) maior, o que se traduz em uma base mais larga para cada uma delas. Nos resultados as porcentagens de erro continuam altas como no caso anterior.

In [ ]:
##Teste 3: ruído = 0.1; sigma = 1000; duas funções de pertinência, 
##Ao adicionar um ruído de 0.1, a função de entrada seno continua com o seu formato, porém
##ao deixar o sigma com um número grande, a porcentagem de erro fica alto e constante.
##E o resultado final não é nada satisfatório, já que "expandimos" muito a "base" da gaussiana
## fazendo com que o formato do seno ficasse perdido
modelo = cria_modelo(1000)
train_data = cria_dados(seno, ruido = 0.1)
train_anfis(modelo, train_data, 100, True) 

### Teste 4: com ruído, base média e cinco partes

Neste caso, um sigma médio foi escolhido ($\sigma = 10$) e cinco funções de pertinência foram adotadas. É possível ver que o modelo foi se estabilizando ao longo do tempo, diferentemente dos dois casos anteriores.

In [ ]:
##Teste 4: ruído = 0.1; sigma = 10; cinco funções de pertinência, 
##Ao adicionar um ruído de 0.1, a função seno continua com o seu formato, porém
##ao deixar o sigma com um número médio, mas agora, aumentando as funções de pertinência,
##temos um caso interessante, onde a função treinada se aproxima um pouco mais da função
##original. 
modelo = cria_modelo(10, [1.0, 2.0, 2.5, 3.20, 7.0])
train_data = cria_dados(seno, ruido = 0.1)
train_anfis(modelo, train_data, 100, True) 